In [1]:
import sys

sys.path.insert(0, '..')

#!{sys.executable} -m pip install folium
#!{sys.executable} -m pip install openapi_core


In [2]:
from morpheus.modflow.types.discretization.spatial import Rotation, LengthUnit, Grid
from morpheus.modflow.types.geometry import Polygon

polygon = Polygon(
  type='Polygon',
  coordinates=[[
    (13.922514437551428, 50.964720483303836),
    (13.925250781947113, 50.965228748412386),
    (13.925036413951403, 50.96623732041704),
    (13.92222441026388, 50.96609040370362),
    (13.922514437551428, 50.964720483303836)
  ]]
)

rotation = Rotation(20)
length_unit = LengthUnit.meters()
x_coordinates = [0, 0.1, 0.21, 0.22, 0.23, 0.24, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
y_coordinates = [0, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 1]

grid = Grid.from_polygon_with_relative_coordinates(
  polygon=polygon,
  x_coordinates=x_coordinates,
  y_coordinates=y_coordinates,
  rotation=rotation,
)
grid

Grid(origin=Point(coordinates=(13.921781413845425, 50.96598885939761), type='Point'), x_distances=[35.55136551768519, 39.106502069453704, 3.555136551768527, 3.5551365517685127, 3.5551365517685127, 3.555136551768527, 17.775682758842592, 35.5513655176852, 35.551365517685184, 35.551365517685184, 35.551365517685184, 35.55136551768521, 35.55136551768521, 35.551365517685156], y_distances=[51.66873376723379, 2.5834366883616866, 2.5834366883616937, 2.5834366883616937, 2.5834366883616795, 2.5834366883616937, 12.917183441808447, 25.834366883616894, 25.834366883616894, 25.834366883616894, 25.834366883616894, 77.50310065085068], rotation=Rotation(value=20), length_unit=LengthUnit(unit=2))

In [3]:
import folium

m = folium.Map(tiles="cartodbpositron", crs="EPSG3857", zoom_start=12, location=[50.965, 13.922])
m.add_child(folium.GeoJson(polygon.as_geojson()))
m.fit_bounds(m.get_bounds())
m

In [4]:
grid_geometry = grid.as_geojson()
m.add_child(folium.GeoJson(grid_geometry.as_geojson()))
m.fit_bounds(m.get_bounds())
m

In [5]:

cell_geometries = grid.get_cell_geometries()

n_x = grid.nx()
n_y = grid.ny()

for x in range(n_x):
  for y in range(n_y):
    cell_geometry = cell_geometries[x][y]
    folium.GeoJson(cell_geometry.as_geojson()).add_to(m)

m


In [6]:
from morpheus.modflow.infrastructure.geometry.utils import calculate_cells_from_polygon, \
  calculate_affected_cells_geometries

affected_cells = calculate_cells_from_polygon(polygon, grid)
affected_cells_geometries = calculate_affected_cells_geometries(affected_cells, grid)

m = folium.Map(tiles="cartodbpositron", crs="EPSG3857", zoom_start=12, location=[50.965, 13.922])
folium.GeoJson(polygon.as_geojson(), name="hello world").add_to(m)
folium.GeoJson(grid_geometry.as_geojson()).add_to(m)
folium.GeoJson(affected_cells_geometries.as_geojson()).add_to(m)
m.fit_bounds(m.get_bounds())
m